# Pre-Processing Twitter Corpus

## étape 0: Load necessary packages 

In [ ]:
# punkt package
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import re
import nltk
import contractions
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer


NameError: name 'nltk' is not defined

In [ ]:
# for expanding contraction words e.g. isn't --> is not
# !pip install contractions
!pip install seaborn

In [ ]:


def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet) # remove Twitter links
    return tweet

def remove_tags(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hash tags
    tweet = re.sub('RT @[\w_]+:','', tweet)  # remove retweet label
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove tweeted at
    return tweet

punctuation = '!”$%&\"’()*+,-./:;<=>?[\\]^_`{|}~•@'
def remove_nonText(tweet):
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub(r'\n','', tweet)  # remove escape sequence
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet = re.sub('📝 …', '', tweet) # un exemple d'image que vous pouvez compléter !
    return tweet

def remove_contraction(tweet): #enlever les contractions ou les abréviations
    # tweet = ' '.join([contractions.fix(word) for word in tweet.split()])
    contraction_dict = {"t'es" : "tu es", "c'est": "ce est", "c":"ce est"} # ajouter d'autres si nécessaire
    tweet = " ".join([contraction_dict.get(i,i) for i in tweet.split()])
    return tweet

def tokenize(text):
    tknzr = TweetTokenizer(reduce_len=True) 
    return tknzr.tokenize(text)

def remove_stopwords(text):
    stop_words = set(stopwords.words('french'))
    return " ".join([token for token in text if token.lower() not in stop_words])


def preprocess_tweet(tweet):
    """Main master function to clean and normalizing tweets, and tokenizing use lemmatization"""
    tweet = remove_links(tweet)
    tweet = remove_tags(tweet)
    tweet = remove_users(tweet)
    tweet = remove_nonText(tweet)
    tweet = tweet.lower()  # lower case
    tweet = remove_contraction(tweet)

    ######################################

    tweet = tokenize(tweet)  # apply tokenization
    tweet = remove_stopwords(tweet)

    return tweet

In [ ]:
df = pd.read_csv('../Semaine6/tweets_Greve_2023.csv',encoding='utf-8')
df_fr = df[df['Langue'] == 'fr'].copy()

df_fr['Normalized'] = [preprocess_tweet(sentence) for sentence in df_fr['Tweet']]
print(df_fr.head())

all_Normalized_words = ' '.join([word for word in df_fr['Normalized']])
tokenized_Normalized_words = word_tokenize(all_Normalized_words)
tokenized_Normalized_words = tokenize(all_Normalized_words)

In [ ]:
# lets find the most frequent words
from nltk.probability import FreqDist
fdist_normalized = FreqDist(tokenized_Normalized_words)

print("The most frequent words in the corpus---- after normalizing the data!")
print(fdist_normalized.most_common(20))

fdist_normalized.plot(50)

#### Loi de Zipf
Les distributions comme celles présentées ci-dessus sont typiques du langage. En réalité, ces distributions en longue traîne sont si courantes dans n'importe quel corpus de langage naturel (un livre, un ensemble de textes d'un site web ou des paroles enregistrées) que la relation entre la fréquence d'utilisation d'un mot et son rang a été largement étudiée. Une version classique de cette relation est connue sous le nom de loi de Zipf, du nom de George Zipf, un linguiste américain du XXᵉ siècle.

https://www.youtube.com/watch?v=fCn8zs912OE&ab_channel=Vsauce

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4176592/

La loi de Zipf stipule que la fréquence d’apparition d’un mot est inversement proportionnelle à son rang. Ainsi, le mot le plus fréquent apparaît deux fois plus souvent que le deuxième mot le plus fréquent, trois fois plus souvent que le troisième, et ainsi de suite, jusqu’au mot le moins fréquent. Plus précisément, un mot situé à la position n apparaît 1/n fois aussi souvent que le mot le plus fréquent.

In [ ]:
# Import Plotting libraries
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
## Creating FreqDist for whole BoW, keeping the 20 most common tokens
all_fdist = fdist_normalized.most_common(20)

## Conversion to Pandas series via Python Dictionary for easier plotting
all_fdist = pd.Series(dict(all_fdist))

## Setting figure, ax into variables
fig, ax = plt.subplots(figsize=(10,7))

## Seaborn plotting using Pandas attributes + xtick rotation for ease of viewing
all_plot = sns.barplot(x=all_fdist.index, y=all_fdist.values, ax=ax)
plt.xticks(rotation=30);
plt.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

stop_words = set(stopwords.words('french'))

wordcloud = WordCloud(width= 1000, height = 600, max_words=200,
                      random_state=1, background_color='White',
                      collocations=False, stopwords = stop_words).generate(all_Normalized_words)
plt.figure(figsize=(5, 5))
plt.imshow(wordcloud) 
plt.axis("off")
plt.show()

### Stemming
Pour réduire la complexité d’un texte, on peut "réduire" les formes dérivées à leur forme la plus simple.

Il existe deux approches dans le domaine :

- la lemmatisation : un lemme correspond à la plus petite unité lexicale faisant sens, qui est souvent l’équivalent à une entrée dans le dictionnaire.
- la racinisation (stemming) plus fruste mais plus rapide. Le terme est dépouillé de ses préfixes et suffixes, ainsi que de toutes ses déclinaisons. Souvent, cette racine ne correspond pas à un “vrai mot”…

Attention il n'existe pas de fonction de lemmatisation de corpus français dans NLTK, un bel objectif pour vos futurs projets !

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='french')

all_Normalized_stemmed = [stemmer.stem(word) for word in tokenized_Normalized_words]

In [ ]:
print(all_Normalized_stemmed[0:50])

In [ ]:
# lets find the most frequent words
fdist_stemm = FreqDist(all_Normalized_stemmed)

In [ ]:
print("The most frequent words in the corpus---- after normalizing the data!")
print(fdist_normalized.most_common(20))
print("The most frequent words in the corpus---- after stemming the data!")
print(fdist_stemm.most_common(20))

In [ ]:
from nltk.stem.snowball import FrenchStemmer
stemmer2 = FrenchStemmer()
all_Normalized_stemmed2 = [stemmer2.stem(word) for word in tokenized_Normalized_words]
fdist_stemm = FreqDist(all_Normalized_stemmed)

In [ ]:
print("The most frequent words in the corpus---- after normalizing the data!")
print(fdist_normalized.most_common(20))
print("---------------------------------")
print("The most frequent words in the corpus---- after stemming the data!")
print(fdist_stemm.most_common(20))
print("---------------------------------")
print("The most frequent words in the corpus---- after stemming the data with French Stemmer")
print(fdist_stemm.most_common(20))

In [ ]:
#Lemmatization in Engligh in NLTK
from nltk.stem import WordNetLemmatizer
# create an object of class WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize("takes", 'v'))
print(lemmatizer.lemmatize("taken", 'v'))
print(lemmatizer.lemmatize("take", 'v'))
print(lemmatizer.lemmatize("taking", 'v'))

#### Librarie Spacy 
SpaCy est une bibliothèque Python open source pour le traitement du texte et des langues naturelles. Créé par l’équipe de la startup Explosion AI, spaCy a été publiée sous la licence MIT en 2015 et ses principaux développeurs sont Matthew Honnibal et Ines Montani, les fondateurs de la startup.
- Contrairement à NLTK, qui est largement utilisé pour l’enseignement et la recherche, spaCy est conçu pour l’utilisation de production sur de grandes quantités de textes avec une excellente performance et précision
- Il prend en charge des modèles statistiques pour 21 langues dont le français, l’anglais, l’allemand, l’espagnol, l’italien, le portugais et le néerlandais. 
- Vous pouvez écouter le tuto de Clément Plancq ! https://github.com/clement-plancq/tuto-mate

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("Z58g33GglR0")

In [ ]:
# installer Spacy dans terminal
#!conda install -c conda-forge spacy
#!pip install -U spacy

In [ ]:
#!python3 -m spacy download fr_core_news_md
#!conda install -c conda-forge spacy-model-fr_core_news_md

In [ ]:
import spacy
from spacy.lang.fr.examples import sentences 

nlp = spacy.load("fr_core_news_md")

In [ ]:
len(all_Normalized_words[0:5000])

In [ ]:
doc = nlp(all_Normalized_words[0:7000])

In [ ]:
for token in doc[0:15]:
    print((token.text, token.lemma_, token.pos_, token.tag_))

In [ ]:
print(doc[2].text, doc[2].morph)  # 'Case=Nom|Number=Sing|Person=2|PronType=Prs'
print(doc[2].text, doc[2].pos_) # 'PRON'

In [ ]:
all_Normalized_lemma = [token.lemma_ for token in doc]

In [ ]:
fdist_lemma = FreqDist(all_Normalized_lemma)

In [ ]:
print("The most frequent words in the corpus---- after normalizing the data!")
print(fdist_normalized.most_common(20))
print("---------------------------------")
print("The most frequent words in the corpus---- after stemming the data!")
print(fdist_stemm.most_common(20))
print("---------------------------------")
print("The most frequent words in the corpus---- after lemmatizing the data with spacy lemma")
print(fdist_lemma.most_common(50))

In [ ]:
all_lemmas_words = ' '.join([word for word in all_Normalized_lemma])

In [ ]:
wordcloud = WordCloud(width= 1000, height = 600, max_words=500,
                      random_state=1, background_color='White',
                      collocations=False).generate(all_lemmas_words)
plt.figure(figsize=(10, 10))
plt.imshow(wordcloud) 
plt.axis("off")
plt.show()

In [ ]:
import seaborn as sns
sns.set_theme(style="ticks")

# Obtain top 10 words
top_20 = fdist_lemma.most_common(50)

# Create pandas series to make plotting easier
fdist = pd.Series(dict(top_20))

sns.barplot(y=fdist.index, x=fdist.values, color='red');
plt.show()

# 3 - Keywords Extraction with TF-IDF
La recherche de "Mots-clés – fouille de textes" consiste à identifier les termes ou concepts les plus récurrents dans un ensemble de données. Ceci peut s’avérer très utile pour analyser les avis de clients ou les conversations sur les réseaux sociaux...

## 1 - TF-IDF : qu’est-ce que c’est ?

TF-IDF acronyme de « Terme Frequency - Inverse Document Frequency ». Il s’agit d’un score qui permet de quantifier l’importance d’un terme pour un document particulier compte tenu de ses occurrences dans tous les autres documents. De fait, un terme est d’autant plus pertinent s’il est surreprésenté dans un document, tout en étant rare dans l’ensemble du corpus. Il s’agit de la méthode utilisée par les moteurs de recherches sur internet (ex : Google) pour répondre à une recherche de mots clés de ses utilisateurs. TF-IDF est aujourd’hui la référence pour l’analyse de textes mal structurés (tweets, dialogue tchat...). Ce score est calculé à partir de deux composantes :
- 1) TF : Une fonction croissante de la fréquence du terme dans le document à l’étude.

--->    Nombre d’occurrence du terme analysé / Nombre de termes total.
- 2) IDF : Une fonction inversement proportionnelle à la fréquence du terme dans le corpus.

--->    Log(Nombre total de documents / Nombre de documents contenant le terme analysé)

- Le score TF-IDF est le résultat de la multiplication des deux termes TF et IDF.


In [ ]:
# define functions used for TF IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfTransformer

Collection = [ ]
df_tfidf = pd.DataFrame()
Collection = df_fr['Normalized'].to_list()

vectorizer = TfidfVectorizer() #stop_words=stop_words_french)
vectors = vectorizer.fit_transform(Collection)
feature_names = vectorizer.get_feature_names_out()
dense = vectors.todense()
denselist = dense.tolist()
tf_idf_vector = pd.DataFrame(denselist, columns=feature_names)

In [ ]:
tf_idf_vector

In [ ]:
import numpy as np
#Now you can create an inverse mapping:
m = {v: k for (k, v) in vectorizer.vocabulary_.items()}

#and this gives the influential word per doc:
keyword=[m[t] for t in np.array(np.argmax(vectors, axis=1)).flatten()]

In [ ]:
#Display Most Significant Term for each Document¶
keyword[0:10]

In [ ]:
list_tfidf=[]
Top_n=100
for i in range(len(Collection)):
    doc=tf_idf_vector.iloc[i].to_numpy()
    list_tfidf.append([m[t] for t in np.array(np.argsort(-doc)).flatten()[0:Top_n]])
df_tfidf=pd.DataFrame(list_tfidf)

In [ ]:
# Find the 3 top words in each document
top_dict = {}
data = tf_idf_vector.transpose()

for i, c in enumerate(data.columns):
    top = data.loc[:,c].sort_values(ascending=False).head(50)
    top_dict[data.columns[i]]= list(zip(top.index, top.values))

# Print the top 3 words said by tweet
for tweet, top_words in top_dict.items():
    print(tweet)
    print(', '.join([word for word, count in top_words[0:3]]))
    print(', '.join([str(count) for word, count in top_words[0:3]]))
    print('---')

In [ ]:
#Ranking documents by TF-IDF Score for a Search Word¶
query = "grève"
tf_idf_vector.sort_values(by=[query], ascending=True)[query]

In [ ]:
# Get the index of DataFrame
tf_idf_vector.sort_values(by=[query], ascending=False).index[0:10]

In [ ]:
# Définir le mot recherché
query = "grève"
# trié la colonne correspondant à ce terme dans la matrice des scores TF-IDF
index = tf_idf_vector.sort_values(by=[query], ascending=False).index.to_list()
#Utiliser les index pour trouver les Tweets les plus 
for i in range(15):
    print("Tweet #", index[i], ":", df_fr['Tweet'][index[i]])